In [2]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [3]:
bigM = 1000000

In [4]:
# Load data
locations_df = pd.read_csv('locations.csv')
order_list_df = pd.read_excel('order_list_1.xlsx')
travel_matrix_df = pd.read_csv('travel_matrix.csv')
trucks_df = pd.read_csv('trucks.csv')

In [5]:
Q = sorted(list(set(trucks_df['truck_max_weight'])))

In [6]:
Q = [Q[0]]*10 + [Q[1]]*8 + [Q[2]]*10 + [Q[3]]*4 + [Q[4]]*5

In [7]:
dest1 = list(set(order_list_df['Destination Code']))[:15]

In [8]:
dest = [str(i) for i in dest1]

In [9]:
order_list_df = order_list_df[order_list_df['Destination Code'].isin(dest1)]

In [10]:
order_list_df1 = order_list_df.sort_values(by = 'Destination Code').groupby('Destination Code').sum("Total Weight").reset_index()

In [11]:
locations_df = locations_df[locations_df['location_code'].isin(dest + ['A123'])]

In [12]:
# Convert loading/unloading windows to minutes with explicit format
locations_df['start_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.minute
locations_df['end_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.minute

In [13]:
customers = locations_df.sort_values(by = 'location_code').iloc[:len(order_list_df1),:]

In [14]:
locations_df2 = locations_df.sort_values(by = 'location_code')

In [15]:
locations_df2

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
65,10208776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
69,10210421,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
45,10212373,['3 tonner Box'],8:00,22:00,480,1320
90,10219625,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
74,10310757,['3 tonner Box'],8:00,22:00,480,1320
55,10366979,['3 tonner Box'],8:00,22:00,480,1320
101,11950159,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
50,12219936,['3 tonner Box'],8:00,22:00,480,1320
129,12231175,['3 tonner Box'],8:00,22:00,480,1320
66,12475516,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [16]:
cap_df = dict(zip(trucks_df['truck_type'], trucks_df['truck_max_weight']))

In [17]:
max_veh_access = []
for i in locations_df2.index:
    max_veh_access.append(cap_df[eval(locations_df2['trucks_allowed'][i])[-1]])

In [18]:
max_veh_access = max_veh_access[len(order_list_df1):] + max_veh_access[:len(order_list_df1)]

In [19]:
max_veh_access

[17000,
 7000,
 9200,
 2800,
 9200,
 2800,
 2800,
 7000,
 2800,
 2800,
 7000,
 2800,
 9200,
 2800,
 17000,
 9200]

In [20]:
depot = locations_df.sort_values(by = 'location_code').iloc[len(order_list_df1):,:]

In [21]:
Nodes = pd.concat([depot,customers], ignore_index = True)

In [22]:
Nodes

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
0,A123,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,18:00,480,1080
1,10208776,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
2,10210421,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
3,10212373,['3 tonner Box'],8:00,22:00,480,1320
4,10219625,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
5,10310757,['3 tonner Box'],8:00,22:00,480,1320
6,10366979,['3 tonner Box'],8:00,22:00,480,1320
7,11950159,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
8,12219936,['3 tonner Box'],8:00,22:00,480,1320
9,12231175,['3 tonner Box'],8:00,22:00,480,1320


In [23]:
vehicles = [ k for k in range(0,len(Q))]
customers = [ i for i in range(1,len(Nodes))]
nodes = [ i for i in range(0, len(Nodes))]

In [24]:
sum(Q)

257400

In [25]:
demand = [0] + list(order_list_df1['Total Weight'])

In [26]:
sum(demand)

13924.0

In [27]:
start_time = list(Nodes['start_minutes'])
finish_time = list(Nodes['end_minutes'])

In [28]:
# Constants
service_time_customer = 20  
service_time_depot = 60 

In [29]:
dest2 = ['A123'] + sorted(dest)

In [30]:
dest3 = {} 
for i in range(len(dest2)):
    dest3[dest2[i]] = i

In [31]:
dest3

{'A123': 0,
 '10208776': 1,
 '10210421': 2,
 '10212373': 3,
 '10219625': 4,
 '10310757': 5,
 '10366979': 6,
 '11950159': 7,
 '12219936': 8,
 '12231175': 9,
 '12475516': 10,
 '12475523': 11,
 '12489344': 12,
 '12546176': 13,
 '12779571': 14,
 '13015040': 15}

In [32]:
travel_matrix_df = travel_matrix_df[(travel_matrix_df['source_location_code'].isin(dest + ['A123'])) & (travel_matrix_df['destination_location_code'].isin(dest + ['A123']))]

In [33]:
travel_matrix_df['mapped_source'] = travel_matrix_df['source_location_code'].map(dest3)
travel_matrix_df['mapped_destination'] = travel_matrix_df['destination_location_code'].map(dest3)

In [34]:
dist_matrix = {}
time_matrix = {}
for i in travel_matrix_df.index:
    dist_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_distance_in_km'][i]
    time_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_time_in_min'][i]

In [35]:
start_time

[480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480,
 480]

In [36]:
finish_time

[1080,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320,
 1320]

In [37]:
nodes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [38]:
customers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [39]:
vehicles

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36]

In [40]:
#Decision Variable

my_model=gp.Model('CVRPTW')

xijk=my_model.addVars(nodes, nodes, vehicles, vtype=GRB.BINARY, name='xijk')
sik = my_model.addVars(nodes, vehicles, vtype = GRB.CONTINUOUS, name = 'sik',lb = 0)
q = my_model.addVars(customers, vehicles, vtype = GRB.CONTINUOUS, name = 'qjk', lb = 0)
I = my_model.addVars(vehicles, vtype = GRB.BINARY, name = 'I')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-24


In [41]:
#Objective function
obj_fn = (gp.quicksum(dist_matrix[i,j]* gp.quicksum(xijk[i,j,k] for k in vehicles) for i in nodes for j in nodes) +
          gp.quicksum(I[k] for k in vehicles)
         )

my_model.setObjective(obj_fn, GRB.MINIMIZE)


In [42]:
#Source to sink constraints
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers)<=1 for k in vehicles);

In [43]:
my_model.addConstrs(gp.quicksum(xijk[i,0,k] for i in customers)<=1 for k in vehicles);

In [44]:
# my_model.addConstr(gp.quicksum(xijk[0,j,k] for j in customers for k in vehicles)<=10);

In [45]:
my_model.addConstrs(gp.quicksum(xijk[i,j,k] for i in nodes)- gp.quicksum(xijk[j,i,k] for i in nodes)==0 
                    for j in nodes for k in vehicles);

In [46]:
#capacity constraints
my_model.addConstrs(gp.quicksum(q[j,k] for k in vehicles) == demand[j] 
                    for j in customers); 
my_model.addConstrs(gp.quicksum(q[j,k] for j in customers) <= Q[k] for k in vehicles);

In [47]:
my_model.addConstrs(q[j,k] <= Q[k] * gp.quicksum(xijk[i,j,k] for i in nodes) for j in customers for k in vehicles);

In [48]:
my_model.addConstrs(sik[i,k] + time_matrix[i,j] + service_time_customer
                    - sik[j,k] <= (1-xijk[i,j,k]) *1440
                    for i in customers 
                    for j in customers for k in vehicles);

In [49]:
my_model.addConstrs(sik[i,k] <= finish_time[i] for i in nodes for k in vehicles)
                    
my_model.addConstrs(sik[i,k] >= start_time[i] for i in nodes for k in vehicles);                                   

In [50]:
for i in nodes:
    for j in nodes:
        for k in vehicles:
            if Q[k] <= max_veh_access[j] and Q[k] <= max_veh_access[i]:
                my_model.addConstr(xijk[i,j,k] <= 1);
            else:
                my_model.addConstr(xijk[i,j,k] == 0);

In [51]:
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers) <= I[k] for k in vehicles);

In [53]:
my_model.setParam('Heuristics', 0.6)  

Set parameter Heuristics to value 0.6


In [ ]:
my_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 20291 rows, 10656 columns and 64528 nonzeros
Model fingerprint: 0x3b753f0e
Variable types: 1147 continuous, 9509 integer (9509 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]


Presolve removed 16331 rows and 6192 columns
Presolve time: 0.10s
Presolved: 3960 rows, 4464 columns, 22283 nonzeros
Variable types: 633 continuous, 3831 integer (3831 binary)
Found heuristic solution: objective 1195.5400000

Root relaxation: objective 2.802800e+02, 702 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  280.28000    0   28 1195.54000  280.28000  76.6%     -    0s
H    0     0                    1144.4400000  280.28000  75.5%     -    0s
H    0     0                    1063.0800000  280.28000  73.6%     -    0s
     0     0  280.28000    0   28 1063.08000  280.28000  73.6%     -    0s
H    0     0                     757.1100000  280.28000  63.0%     -    0s
     0     0  280.28000    0   30  757.11000  280.28000  63.0%     -    0s
     0     0  280.28000    0   28  757.11000  280.28000  63.0%     -    0s
     0   

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 159440 104813 infeasible   74       510.76000  370.29825  27.5%  30.1  910s


In [225]:
# Create a new dictionary to store non-zero values
non_zero_xijk = {}

# Iterate over all keys and values in xijk
for (i, j, k), value in xijk.items():
    # Check if the value is a Gurobi variable
    if hasattr(value, "X"):  # If it's a Gurobi variable, use .X to get the optimized value
        if value.X > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value.X
    else:  # Otherwise, assume it's a numeric value
        if value > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value

# Print the non-zero values
print("Non-zero values of x:")
for (i, j, k), value in non_zero_xijk.items():
    print(f"x_{i}_{j}_{k} = {value}")


Non-zero values of x:
x_0_2_29 = 1.0
x_0_8_2 = 1.0
x_0_10_5 = 1.0
x_0_11_0 = 1.0
x_0_14_3 = 1.0
x_0_14_8 = 1.0
x_0_15_23 = 1.0
x_1_0_5 = 1.0
x_1_11_8 = 1.0
x_2_0_29 = 1.0
x_3_0_2 = 1.0
x_4_0_23 = 1.0
x_4_13_5 = 1.0
x_5_3_2 = 1.0
x_6_0_0 = 1.0
x_7_10_0 = 1.0
x_8_5_2 = 1.0
x_9_1_5 = 1.0
x_10_1_8 = 1.0
x_10_4_5 = 1.0
x_10_6_0 = 1.0
x_11_0_8 = 1.0
x_11_7_0 = 1.0
x_11_9_5 = 1.0
x_12_10_8 = 1.0
x_13_11_5 = 1.0
x_14_0_3 = 1.0
x_14_12_8 = 1.0
x_15_4_23 = 1.0


In [226]:
# Create a dictionary to store routes for each vehicle
vehicle_routes = {}

# Iterate over the non_zero_xijk dictionary to create routes
for (i, j, k), value in non_zero_xijk.items():
    if value == 1.0:
        if k not in vehicle_routes:
            vehicle_routes[k] = []  # Create a list for each vehicle if not already created
        vehicle_routes[k].append((i, j))

# Display the routes for each vehicle
for vehicle, route in vehicle_routes.items():
    print(f"Vehicle {vehicle} Route: {route}")

Vehicle 29 Route: [(0, 2), (2, 0)]
Vehicle 2 Route: [(0, 8), (3, 0), (5, 3), (8, 5)]
Vehicle 5 Route: [(0, 10), (1, 0), (4, 13), (9, 1), (10, 4), (11, 9), (13, 11)]
Vehicle 0 Route: [(0, 11), (6, 0), (7, 10), (10, 6), (11, 7)]
Vehicle 3 Route: [(0, 14), (14, 0)]
Vehicle 8 Route: [(0, 14), (1, 11), (10, 1), (11, 0), (12, 10), (14, 12)]
Vehicle 23 Route: [(0, 15), (4, 0), (15, 4)]


In [227]:
max_veh_access

[17000,
 7000,
 9200,
 2800,
 9200,
 2800,
 2800,
 7000,
 2800,
 2800,
 7000,
 2800,
 9200,
 2800,
 17000,
 9200]

In [228]:
non_zero_q = {(i,k):q[i,k].X for i in customers for k in vehicles if q[i,k].X > 0.1}
non_zero_q

{(1, 5): 611.0,
 (2, 29): 2299.0,
 (3, 2): 209.0,
 (4, 23): 2200.0,
 (5, 2): 310.0,
 (6, 0): 304.0,
 (7, 0): 1532.0,
 (8, 2): 311.0,
 (9, 5): 84.0,
 (10, 0): 604.0,
 (11, 5): 350.0,
 (11, 8): 710.0,
 (12, 8): 2090.0,
 (13, 5): 836.0,
 (14, 3): 1254.0,
 (15, 23): 220.0}

In [229]:
demand

[0,
 611.0,
 2299.0,
 209.0,
 2200.0,
 310.0,
 304.0,
 1532.0,
 311.0,
 84.0,
 604.0,
 1060.0,
 2090.0,
 836.0,
 1254.0,
 220.0]

In [230]:
I

{0: <gurobi.Var I[0] (value 1.0)>,
 1: <gurobi.Var I[1] (value -0.0)>,
 2: <gurobi.Var I[2] (value 1.0)>,
 3: <gurobi.Var I[3] (value 1.0)>,
 4: <gurobi.Var I[4] (value -0.0)>,
 5: <gurobi.Var I[5] (value 1.0)>,
 6: <gurobi.Var I[6] (value -0.0)>,
 7: <gurobi.Var I[7] (value -0.0)>,
 8: <gurobi.Var I[8] (value 1.0)>,
 9: <gurobi.Var I[9] (value -0.0)>,
 10: <gurobi.Var I[10] (value 0.0)>,
 11: <gurobi.Var I[11] (value 0.0)>,
 12: <gurobi.Var I[12] (value 0.0)>,
 13: <gurobi.Var I[13] (value 0.0)>,
 14: <gurobi.Var I[14] (value 0.0)>,
 15: <gurobi.Var I[15] (value 0.0)>,
 16: <gurobi.Var I[16] (value 0.0)>,
 17: <gurobi.Var I[17] (value 0.0)>,
 18: <gurobi.Var I[18] (value 0.0)>,
 19: <gurobi.Var I[19] (value 0.0)>,
 20: <gurobi.Var I[20] (value 0.0)>,
 21: <gurobi.Var I[21] (value 0.0)>,
 22: <gurobi.Var I[22] (value 0.0)>,
 23: <gurobi.Var I[23] (value 1.0)>,
 24: <gurobi.Var I[24] (value 0.0)>,
 25: <gurobi.Var I[25] (value 0.0)>,
 26: <gurobi.Var I[26] (value 0.0)>,
 27: <gurobi.Var

In [231]:
#use dest3 for reverse mapping into location codes from index